In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Function to load and preprocess the images and masks
def load_images_and_masks(images_dir, masks_dir):
    images = []
    masks = []
    for image_file in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_file)
        mask_file = os.path.splitext(image_file)[0] + "_mask.png"
        mask_path = os.path.join(masks_dir, mask_file)
        
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize the image and mask to a common size (adjust as needed)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))
        
        # Normalize the pixel values to the range [0, 1]
        image = image / 255.0
        mask = mask / 255.0
        
        images.append(image)
        masks.append(mask)
    
    images = np.array(images)
    masks = np.array(masks)
    
    return images, masks

# Specify the directories for annotated images and masks
images_dir = "img/not_labeled_stones"
masks_dir = "masks/"

# Load and preprocess the images and masks
images, masks = load_images_and_masks(images_dir, masks_dir)

# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(256 * 256, activation='sigmoid'))  # Output layer with sigmoid activation for binary masks
model.add(layers.Reshape((256, 256, 1)))  # Reshape the output to match the mask dimensions

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set
model.fit(train_images, train_masks, epochs=10, batch_size=16, validation_data=(val_images, val_masks))

Epoch 1/10
6/6 [==============================] - 69s 143ms/step - loss: 0.6202 - accuracy: 0.7167 - val_loss: 0.1230 - val_accuracy: 0.9844
Epoch 2/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1213 - accuracy: 0.9885 - val_loss: 0.2043 - val_accuracy: 0.9850
Epoch 3/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1521 - accuracy: 0.9881 - val_loss: 0.1893 - val_accuracy: 0.9832
Epoch 4/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1307 - accuracy: 0.9876 - val_loss: 0.1602 - val_accuracy: 0.9857
Epoch 5/10
6/6 [==============================] - 0s 41ms/step - loss: 0.1142 - accuracy: 0.9887 - val_loss: 0.1405 - val_accuracy: 0.9859
Epoch 6/10
6/6 [==============================] - 0s 41ms/step - loss: 0.0927 - accuracy: 0.9888 - val_loss: 0.1035 - val_accuracy: 0.9859
Epoch 7/10
6/6 [==============================] - 0s 41ms/step - loss: 0.0752 - accuracy: 0.9888 - val_loss: 0.0866 - val_accuracy: 0.9859
Epoch 8/10
6/6 [=========

In [2]:
#save model
model.save('model3.h5')